# Preprocessing satellite images

In this notebook, we will show how to utilize both SpaceNet satellite image and preprocess using white balancing and converting data to 8-bit (0-255)

## Setup

Before proceeding, make sure this notebook connects with the proper kernel (`conda_[name]`, `[name]` is name of the new Conda environment you just created).

In [ ]:
import os, skimage
import numpy as np
from skimage import img_as_ubyte

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

### Plotting preferences

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

The datasets used in this notebook are saved in directory `./data/raw_data/`.

1. **SpaceNet satellite images**:
There are tiles ("chips") of 200m$\times$200m satellite images/labels in this training dataset (`SN2/AOI_2_Vegas`). See [link](https://spacenet.ai/las-vegas/) for more information. We extract the ~0.3m resolution pan-sharpened RGB (`PS-RGB`) 3-channel satellite images and perform white balancing and convert the data to 8-bit. 

In [ ]:
# Paths 
data_dir = './data/raw_data/'

# Prefix of all filename - naming convention
prefix = 'RGB-PanSharpen_AOI_2_Vegas_'

In [ ]:
# Function to whitebalance data and convert data to 8 bit
def percentile_whitebalance(image, percentile_value=99):
    whitebalanced = img_as_ubyte(
            (image*1.0 / np.percentile(image, 
             percentile_value, axis=(0, 1))).clip(0, 1))
    return whitebalanced


In [ ]:
# Scaling values of images between [0,1]
def scale_values(image,clip_min=0, clip_max=1):
    min_value = image.min()
    max_value = image.max()
    scaled_image = ((image - min_value)/(max_value - min_value))*(clip_max - clip_min) + clip_min 
    return scaled_image

In [ ]:
# Show a sample data
sample = 'img1423' # chip ID, img? format

# Read in 4-channel image from GeoTIFF.
img_file = prefix + sample + '.tif'
img_path = os.path.join(data_dir, img_file)
img = skimage.io.imread(img_path)
# rgb = img[..., :3]
# # elev = img[..., -1]
white_bal_img = percentile_whitebalance(img)

# Display satellite image and LiDAR eleveation attribute.
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(scale_values(img))
ax[0].set_title('Pan-sharpened 3-band RGB')
ax[1].imshow(white_bal_img)
ax[1].set_title('White balanced image')
fig.show()

The output from white balancing are combined with the LiDAR data and used in the vegetation-management-remars2022/remars2022-workshop/machine_learning_for_trees_identification.ipynb